# Imports

In [1]:
import sqlite3
from prettytable import from_db_cursor
from project_paths import EMPLOYEES_DB

# Connect to database

In [2]:
con = sqlite3.connect(EMPLOYEES_DB)
cur = con.cursor()

# Source table

In [3]:
cur.execute("""
    SELECT      *
    FROM        tSalary
""")

In [4]:
res = from_db_cursor(cur)
res.align = "l"
print(res)

+--------------+-------------+------------+--------+
| CalendarDate | Employee    | Department | Salary |
+--------------+-------------+------------+--------+
| 20190101     | Employee #1 | Sales      | 1000   |
| 20190101     | Employee #2 | Sales      | 2000   |
| 20190101     | Employee #3 | Marketing  | 3000   |
| 20190201     | Employee #1 | Sales      | 1200   |
| 20190201     | Employee #2 | Sales      | 2200   |
| 20190201     | Employee #3 | Marketing  | 3200   |
| 20190301     | Employee #1 | Sales      | 1500   |
| 20190301     | Employee #2 | Sales      | 2500   |
| 20190301     | Employee #3 | Marketing  | 3500   |
| 20190401     | Employee #1 | Sales      | 1700   |
| 20190401     | Employee #2 | Sales      | 2700   |
| 20190401     | Employee #3 | Marketing  | 3700   |
+--------------+-------------+------------+--------+


# Solution

In [5]:
cur.execute("""
SELECT      CalendarDate,
            Employee,
            Department,
            Salary,
            FIRST_VALUE(Salary) OVER(PARTITION BY Employee ORDER BY CalendarDate asc) as FirstSalary,
            LAG(Salary, 1) OVER(PARTITION BY Employee ORDER BY CalendarDate asc) as PreviousSalary,
            LEAD(Salary, 1) OVER(PARTITION BY Employee ORDER BY CalendarDate asc) as NextSalary,
            SUM(Salary) OVER(PARTITION BY CalendarDate, Department) as SumOfDepartmentSalary,
            SUM(Salary) OVER(PARTITION BY Department ORDER BY CalendarDate asc) as CumulativeValueOfDepartmentSalary
FROM        tSalary
ORDER BY    Employee, CalendarDate
""")

In [6]:
res = from_db_cursor(cur)
res.align = "l"
print(res)

+--------------+-------------+------------+--------+-------------+----------------+------------+-----------------------+-----------------------------------+
| CalendarDate | Employee    | Department | Salary | FirstSalary | PreviousSalary | NextSalary | SumOfDepartmentSalary | CumulativeValueOfDepartmentSalary |
+--------------+-------------+------------+--------+-------------+----------------+------------+-----------------------+-----------------------------------+
| 20190101     | Employee #1 | Sales      | 1000   | 1000        | None           | 1200       | 3000                  | 3000                              |
| 20190201     | Employee #1 | Sales      | 1200   | 1000        | 1000           | 1500       | 3400                  | 6400                              |
| 20190301     | Employee #1 | Sales      | 1500   | 1000        | 1200           | 1700       | 4000                  | 10400                             |
| 20190401     | Employee #1 | Sales      | 1700   | 1000 

# Close connection to database

In [7]:
cur.close()